# This script processes the rev1 results 

1. Compile rev1 with:
```
make bin/rev1
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_rev1.py
```
3. These are the outputs of the benchmarking script that will be processed, and should be already in the `Results` directory:
```
rev1_<hostname>.txt
rev1_static_<hostname>.txt
rev1_guided_<hostname>.txt
rev1_collapse_<hostname>.txt 
rev1_collapse_static_<hostname>.txt
rev1_collapse_guided_<hostname>.txt
rev1_tasks_<hostname>.txt
```
4. Additionally, to compare the results, this notebook also needs the results from O1 and O2:
```
o1_qtree_<hostname>.txt
o2_partree_<hostname>.txt
```

In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the files exist
file_list = [                   
                f'rev1_{hostname}.txt', f'rev1_collapse_{hostname}.txt', # dynamic
                f'rev1_static_{hostname}.txt', f'rev1_collapse_static_{hostname}.txt', # static
                f'rev1_guided_{hostname}.txt', f'rev1_collapse_guided_{hostname}.txt', # guided
                f'rev1_tasks_{hostname}.txt', # tasks
            ]

for rev1_file in file_list:
    # if it is not in Results
    if not os.path.exists(rev1_file):
        # is the file already in the directory?
        assert os.path.exists(os.path.join(hostname, rev1_file)), f'File {rev1_file} not found: something went wrong with the baseline benchmark.'
    # if it is in Results
    else:
        # copy the file to the directory
        assert os.system(f'mv {rev1_file} {hostname}/') == 0, f'Failed to move {rev1_file} to {hostname}/'
# add the path to all the names in the list
for i in range(len(file_list)):
    file_list[i] = os.path.join(hostname, file_list[i])

# Get the results from the `rev1_collapse` file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean
from common.utils import get_nprocs, get_best_level

cloud_list = ['AlcoyH','ArzuaH','BrionFH','BrionUH']

# number of threads used in the execution
num_threads = get_nprocs()
# max number of threads
maxth = max(num_threads) # num_threads[-1]

def tokenize_rev1(filename, noH=False):
    """
        This function reads the output of the benchmark and returns a dictionary with the results.

        Args: 
            filename: the name of the file to read
            noH: if True, remove the 'H' at the end of the cloud names

        Returns:
            A dictionary with the results of the benchmark: dict(cloud_name: dict(chunk_size: dict(num_threads: list(times)))
    """
    experiment ={}
    with open(filename) as f:
        # for each line in the file
        for line in f:
            tokens = line.split()
            # if the line contains the word "Running:"
            if "Running:" in tokens:
                # get the name of the cloud
                name=tokens[2].split("/")[3]
                if noH:
                    name = name[:-1]
                # get the number of threads used
                nth=int(tokens[6])
                # get the chunk size used in the dynamic scheduling of the stage 1
                chunk=int(tokens[8])
                # if the cloud is not in the dictionary, add it
                if name not in experiment:
                    experiment[name]={}
                # if the chunk size is not in the dictionary, add it
                if chunk not in experiment[name]:
                    experiment[name][chunk]={}
                # if the number of threads is not in the dictionary, add it
                if nth not in experiment[name][chunk]:
                    experiment[name][chunk][nth]={}
                    # initialize the list of stage times
                    experiment[name][chunk][nth]['stages']=[]
            # get the time of building the quadtree
            elif 'Quadtree' in tokens:
                experiment[name][chunk][nth]['qtree'] = float(tokens[5])
            # get the time of the stage 1, 2 and 3
            elif "STAGE" in tokens:
                experiment[name][chunk][nth]['stages'].append(float(tokens[5]))
            # get the average time for OWM
            elif 'Average:' in tokens:
                experiment[name][chunk][nth]['owm'] = float(tokens[1])

    results = {}

    # for each cloud
    for cloud in experiment:
        results[cloud] = {}
        # for each chunk size
        for chunk in experiment[cloud]:
            results[cloud][chunk] = {}
            # for each number of threads
            for nth in experiment[cloud][chunk]:
                results[cloud][chunk][nth] = {}
                # the first element in the list is the time of building the qtree
                results[cloud][chunk][nth]['qtree'] = experiment[cloud][chunk][nth]['qtree']
                # the next three elements are the time of the stages 1, 2 and 3; one time for each of three elements
                results[cloud][chunk][nth]['stage1'] = mean(experiment[cloud][chunk][nth]['stages'][0::3])
                results[cloud][chunk][nth]['stage2'] = mean(experiment[cloud][chunk][nth]['stages'][1::3])
                results[cloud][chunk][nth]['stage3'] = mean(experiment[cloud][chunk][nth]['stages'][2::3])
                # the last elment is the average time
                results[cloud][chunk][nth]['owm'] = experiment[cloud][chunk][nth]['owm']
                # keep total = qtree + owm
                results[cloud][chunk][nth]['total'] = results[cloud][chunk][nth]['qtree'] + results[cloud][chunk][nth]['owm']
    return results

# dictionay with the results of the benchmark
all_results = {}
# get the results
for file in file_list:
    # get the key from the filename
    all_results[file.replace(f'_{hostname}.txt', '').replace(f'{hostname}/','')] = tokenize_rev1(file)
# point to the dynamic+collapse experiment as default
rrev1 = all_results['rev1_collapse']
print(list(all_results.keys()))

# Get the results from O2 for comparison

In [ ]:
# this is function was obtained from process_o2partree_results.ipynb
def tokenize2(filename,noH=False):
    experiment ={}

    with open(filename) as f:
        for line in f:
            tokens = line.split()
            if "Running:" in tokens:
                if noH:
                    name=tokens[3].split("/")[3][:-1]
                else:
                    name=tokens[3].split("/")[3]
                nth=int(tokens[11])
                level=int(tokens[19])
                if name not in experiment:
                    experiment[name]={}
                if level not in experiment[name]:
                    experiment[name][level]={}
                if nth not in experiment[name][level]:
                    experiment[name][level][nth]=[]
            if "STAGE" in tokens:
                experiment[name][level][nth].append(float(tokens[5]))
            # in O2 output, the quadtree time is after the stages times
            if 'Quadtree' in tokens:
                experiment[name][level][nth].append(float(tokens[5]))
            if 'Average' in tokens:
                experiment[name][level][nth].append(float(tokens[2]))

    results = {}

    for i in experiment:
        results[i]={}
        for j in experiment[i]:
            results[i][j]={}
            for k in experiment[i][j]:
                results[i][j][k]={}
                results[i][j][k]['stage1']=mean(experiment[i][j][k][0:15:3])
                results[i][j][k]['stage2']=mean(experiment[i][j][k][1:15:3])
                results[i][j][k]['stage3']=mean(experiment[i][j][k][2:15:3])
                results[i][j][k]['qtree']=experiment[i][j][k][15]
                results[i][j][k]['owm']=experiment[i][j][k][16]
                # get the total time
                results[i][j][k]['total'] = results[i][j][k]['qtree'] + results[i][j][k]['owm']
    return results

# add the O2 results to the dictionary
all_results['o2'] = tokenize2(f'{hostname}/o2_partree_{hostname}.txt')
# point O2 results
ro2 = all_results['o2']

# Get O1 results for comparison

In [ ]:
# this code was obtained from process_o1quadtree_results.ipynb
experiment ={}
# get the number of processes
num_threads = get_nprocs()
with open(f'{hostname}/o1_qtree_{hostname}.txt') as f:
    for line in f:
        tokens = line.split()
        if "Running:" in tokens:
            name=tokens[2].split("/")[3]
            nth=int(tokens[6])
            if name not in experiment:
                experiment[name]={}
        if 'Quadtree' in tokens:
            experiment[name][nth]=[float(tokens[5])]
        if "STAGE" in tokens:
            experiment[name][nth].append(float(tokens[5]))
        if 'Average:' in tokens:
            experiment[name][nth].append(float(tokens[1]))

results = {}

for i in experiment:
    results[i]={}
    for j in experiment[i]:
        results[i][j]={}
        results[i][j]['qtree']=experiment[i][j][0]
        results[i][j]['stage1']=mean(experiment[i][j][1:15:3])
        results[i][j]['stage2']=mean(experiment[i][j][2:15:3])
        results[i][j]['stage3']=mean(experiment[i][j][3:15:3])
        results[i][j]['owm']=experiment[i][j][16]
        # save the total
        results[i][j]['total'] = results[i][j]['qtree'] + results[i][j]['owm']

# add the O1 results to the dictionary
all_results['o1'] = results
# point to O1 results
ro1 = all_results['o1']

In [ ]:
def plot_chunk_time(nth, results):
    """
        This function plots the time of building the octree, the time of the stage 1, 2 and 3 and the average time for OWM for each cloud and chunk size.

        Args:
            nth: the number of threads used in the execution
            results: the dictionary with the results of the benchmark
    """
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    # for each cloud
    for i,cloud in enumerate(cloud_list):
        # get the list of chunk sizes
        blist = list(results[cloud].keys())
        # plot average time for OWM
        axs[i].plot(np.array(blist), np.array([results[cloud][chunk][nth]['owm'] for chunk in blist]), marks[0], linewidth=linew, markersize=markers)
        # plot the time of building the qtree
        axs[i].plot(np.array(blist), np.array([results[cloud][chunk][nth]['qtree'] for chunk in blist]), marks[1], linewidth=linew, markersize=markers)
        # plot the total time
        axs[i].plot(np.array(blist), np.array([results[cloud][chunk][nth]['total'] for chunk in blist]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(cloud,fontsize=16)
        axs[i].set_xlabel('Chunk Size', fontsize=xlabelfs)
        axs[i].set_xticks(blist)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        axs[i].grid()

    if(nth==1):
        fig.suptitle('Execution time (sec.)',  fontweight='bold', fontsize=18)
    if(nth>1):
        fig.suptitle(f'{nth}-threads execution time (sec.) REV1 @ {hostname.upper()}',  fontweight='bold', fontsize=18)

    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    pp = PdfPages(os.path.join(hostname, f'Rev1_{nth}coresExecTime-{hostname}.pdf'))
    pp.savefig(fig)
    pp.close()

# plot 8 and 16 if AlderLake
if hostname == 'alder':
    plot_chunk_time(8, rrev1)
    plot_chunk_time(16, rrev1)
else:
    plot_chunk_time(num_threads[-1], rrev1)


In [ ]:
def plot_times_index(results, ro2, maxth=8):
    """
        This function plots the average time for OWM, the average time for OWM in the baseline, the time of building the octree and the total time for each cloud and the best chunk size.

        Args:
            results: the dictionary with the results of the benchmark
            ro2: the dictionary with the results for comparison
            maxth: the maximum number of threads used in the execution
    """
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM Rev1', 'OWM O2', 'Tree Const.', 'Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(cloud_list):
        # select the best chunk for each experiment
        bestchunk = get_best_level(results, name, maxth)
        # get the list of number of threads
        num_threads = list(results[name][bestchunk].keys())
        # get the average time for OWM
        axs[i].plot(np.array(num_threads), np.array([results[name][bestchunk][nth]['owm'] for nth in num_threads]), marks[0], linewidth=linew, markersize=markers)
        # get the best level
        bestlevel = get_best_level(ro2, name, maxth)
        # get the average tiem for OWM in the baseline
        axs[i].plot(np.array(num_threads), np.array([ro2[name][bestlevel][nth]['owm'] for nth in num_threads]), '--x', color='red', linewidth=linew, markersize=markers)
        # get the time of building the qtree
        axs[i].plot(np.array(num_threads), np.array([results[name][bestchunk][nth]['qtree'] for nth in num_threads]), marks[1], linewidth=linew, markersize=markers)
        # get the total time
        axs[i].plot(np.array(num_threads), np.array([results[name][bestchunk][nth]['total'] for nth in num_threads]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(f'{name} (chunks={bestchunk},lev={bestlevel})',fontsize=16)
        axs[i].set_xlabel('Num threads', fontsize=xlabelfs)
        axs[i].set_xticks(num_threads)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        axs[i].grid()

    fig.suptitle(f'Execution time (sec.) REV1 @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)

plot_times_index(rrev1, ro2, maxth=num_threads[-1])

In [ ]:
def plot_times_owm(results, maxth=8):
    """
        This function plots the average time for OWM for each cloud and each configuration.
    """
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(cloud_list):
        for config, ri in results.items():
            if config == 'baseline':
                # get the average tiem for OWM in the baseline
                axs[i].plot(np.array(num_threads), np.array([ri[name]['par'][nth]['owm'] for nth in num_threads]), '--x', color='red', linewidth=linew, markersize=markers, label=config)
                continue
            elif config == 'o1':
                continue
            else:
                # select the best chunk/level for each experiment
                bestconf = get_best_level(ri, name, maxth)
                # get the list of number of threads
                num_threads = list(ri[name][bestconf].keys())
                # get the average time for OWM
                axs[i].plot(np.array(num_threads), np.array([ri[name][bestconf][nth]['owm'] for nth in num_threads]), marks[0], linewidth=linew, markersize=markers, label=config)

        axs[i].set_title(f'{name} (chunks={bestconf})',fontsize=16)
        axs[i].set_xlabel('Num threads', fontsize=xlabelfs)
        axs[i].set_xticks(num_threads)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        axs[i].grid()

    fig.suptitle(f'Execution time (sec.) REV1 @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)

plot_times_owm(all_results, maxth=num_threads[-1])

# Compare OWM traversal improvement

In [ ]:
def printowm(nth, res, rcmp):
    """
        This function prints the best chunk size for each cloud, the best time and cofig. for O2 and the speedup of OWM-rev1_XX vs OWM-O2.

        Args:
            nth(int): the number of threads used in the execution
            res(dict(results)): the dictionary with the results of the benchmark
            rcmp(results): the dictionary with the results for comparison
    """
    slist = {} # speedup dict
    for cloud in rcmp:
        bestlevel = get_best_level(rcmp, cloud, nth)
        print("Cloud {} has {}-cores OWM-O2 time = {:.2f} @ level={}.".format(cloud, nth, rcmp[cloud][bestlevel][nth]['owm'], bestlevel))    
        for config,ri in res.items():
            # avoid comparing these results as they do not apply in this case
            if config in ['o1', 'o2']:
                continue
            if config not in slist:
                slist[config] = []
            bestchunk = get_best_level(ri, cloud, nth)
            speedup = rcmp[cloud][bestlevel][nth]['owm']/ri[cloud][bestchunk][nth]['owm']
            print(f"Cloud {cloud} has {nth}-cores OWM-{config} time = {ri[cloud][bestchunk][nth]['owm']:.2f} @ chunk={bestchunk}. Speedup ({config} vs O2) = {speedup:.2f}x.")    
            slist[config].append(speedup)
        print()
    # print a statistical summary of the speedup
    for config, s in slist.items():
        print(f"Speedup ({config} vs O2) for {nth}-cores = {np.mean(s):.2f}x +/- {np.std(s):.2f}x")
        print(f'\tMin: {min(s):.2f}x')
        print(f'\tMax: {max(s):.2f}x')

# print the improvement in the OWM time
printowm(num_threads[-1], all_results, ro2)

In [ ]:
import seaborn as sns

#Configuration variables
def plot_res(res, ro1, rcmp, maxth=8):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 10
    linew = 2
    markers = 8

    fig = plt.figure()

    marks=['o-','x-','s-','v-','+-']
    # get the bestchunk for each cloud
    bestchunk = [min(res[i], key=lambda x: res[i][x][maxth]['total']) for i in res]
    # get the list of number of threads
    numthreads = list(res[cloud_list[0]][bestchunk[0]].keys())
    labels=['OWM Trav.','Total']

    # get the color palette
    colors = sns.color_palette("deep")

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 5), constrained_layout=True)
    legend_labels=[i+" (chunk="+str(j)+")" for i,j in zip(res,bestchunk)]
    legend_labels.append('Ideal')

    for subfig,phase in enumerate(['owm']):
        print("Phase: "+phase+ " speedup for each cloud and best chunk for each cloud: "+str(list(res.keys())))
        for (i,z,color) in zip(res,marks,colors[:len(res)]):
            # get the best chunk for dynamic scheduler
            chunk = get_best_level(res, i, maxth)
            print(np.array([res[i][chunk][1][phase]/res[i][chunk][j][phase] for j in numthreads]))
            # plot the speedup of rev1 wrt O1
            axs.plot(np.array(numthreads), np.array([ro1[i][j][phase]/res[i][chunk][j][phase] for j in numthreads]), z, linewidth=linew, markersize=markers, color=color, label=f'{i} (chunk={chunk})')
            # get the best level for cmp
            level = get_best_level(rcmp, i, maxth)
            # plot the speedup of O2 wrt O1
            axs.plot(np.array(numthreads), np.array([ro1[i][j][phase]/rcmp[i][level][j][phase] for j in numthreads]), ':p', linewidth=linew, markersize=markers, color=color, label=f'{i} O2')
            
        # axs.plot(np.array(numthreads), np.array(numthreads), '-', linewidth=linew, markersize=markers, color=colors[6], label='Ideal')
        axs.set_title(labels[subfig], fontsize=titlefs)
        axs.set_xlabel('Number of cores', fontsize=xlabelfs)
        axs.set_xticks(numthreads)
        axs.tick_params(axis='x', labelsize=xticksfs)
        axs.grid()

    fig.suptitle(f'Speedup wrt O1 - REV1 vs O2 @ {hostname.upper()}',  fontweight='bold', fontsize=titlefs)
    axs.set_ylabel('Speedup', fontsize=ylabelfs)
    axs.legend(loc='best', fontsize=legendfs)

    plt.show()
    return fig

fig = plot_res(rrev1, ro1, ro2, num_threads[-1])

In [ ]:
import seaborn as sns

#Configuration variables
def plot_speed_owm(results, maxth=8):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 10
    linew = 2
    markers = 8

    fig = plt.figure()

    marks=['o-','x-','s-','v-','+-']
    # get the color palette
    colors = sns.color_palette("deep")

    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True)
    for i,name in enumerate(cloud_list):
        for k,(config, ri) in enumerate(results.items()):
            if config == 'o2':
                continue
            if config == 'baseline':
                # get the number of threads used in the execution
                numthreads = list(ri[name]['par'].keys())
                axs[i].plot(np.array(numthreads), np.array([ri[name]['seq']['owm']/ri[name]['par'][j]['owm'] for j in numthreads]), ':p', linewidth=linew, markersize=markers, color=colors[k], label=config)
            elif config == 'o1':
                # get the number of threads used in the execution
                numthreads = list(ri[name].keys())
                axs[i].plot(np.array(numthreads), np.array([ri[name][1]['owm']/ri[name][j]['owm'] for j in numthreads]), ':p', linewidth=linew, markersize=markers, color=colors[k], label=config)
            else:
                # get the bestconfig for each cloud
                bestconfig = get_best_level(ri, name, maxth)
                # get the list of number of threads
                numthreads = list(ri[name][bestconfig].keys())
                axs[i].plot(np.array(numthreads), np.array([ri[name][bestconfig][1]['owm']/ri[name][bestconfig][j]['owm'] for j in numthreads]), marks[k%len(marks)], linewidth=linew, markersize=markers, color=colors[k], label=f'{config} {bestconfig}')
            
        axs[i].plot(np.array(numthreads), np.array(numthreads), '-', linewidth=linew, markersize=markers, color=colors[6], label='Ideal')
        axs[i].set_title(f'{name}', fontsize=titlefs)
        axs[i].set_xlabel('Number of cores', fontsize=xlabelfs)
        axs[i].set_xticks(numthreads)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        axs[i].legend(loc='best', fontsize=legendfs)
        axs[i].grid()

    fig.suptitle(f'Speedup REV1 configurations @ {hostname.upper()}',  fontweight='bold', fontsize=titlefs)
    axs[0].set_ylabel('Speedup', fontsize=ylabelfs)
    
    # plt.show()
    return fig

fig = plot_speed_owm(all_results, num_threads[-1])

In [ ]:
pp = PdfPages(os.path.join(hostname, f'Speedup-rev1-collapse-{hostname}.pdf'))
pp.savefig(fig)
pp.close()

# Save results in All_optimizations

In [ ]:
output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")
quadtree_t = []
for i,cloud in enumerate(rrev1):
    bestchunk = get_best_level(rrev1, cloud, maxth, 'total')
    quadtree_t.append(mean(list(rrev1[cloud][bestchunk][j]['qtree'] for j in num_threads)))
    print("Opt1-REV1-Quadtree; {}; {:.5f}; {:.5f};{};{}".format(cloud, quadtree_t[i], rrev1[cloud][bestchunk][maxth]['owm'], bestchunk, 0))
    f.write("Opt1-REV1-Quadtree;{};{:.5f};{:.5f};{};{}\n".format(cloud, quadtree_t[i], rrev1[cloud][bestchunk][maxth]['owm'], bestchunk, 0))

f.close()